In [23]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
#import keras.backend.tensorflow_backend as K
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras.models import Model
from keras import losses

In [24]:
#dir = './sequence/*'
dir = './latest_sequence/bfs/*'

In [38]:
# file read
all_data = []
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        print(f[1:-2])
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

1, 2, 18.33
1, 3, 8.79
2, 3, 13.29
2, 4, 18.91
2, 5, 19.29
3, 6, 9.82
2, 1, 18.33
2, 3, 13.29
2, 4, 18.91
2, 5, 19.29
1, 3, 8.79
3, 6, 9.82
3, 1, 8.79
3, 2, 13.29
3, 6, 9.82
1, 2, 18.33
2, 4, 18.91
2, 5, 19.29
4, 2, 18.91
2, 1, 18.33
2, 3, 13.29
2, 5, 19.29
1, 3, 8.79
3, 6, 9.82
5, 2, 19.29
2, 1, 18.33
2, 3, 13.29
2, 4, 18.91
1, 3, 8.79
3, 6, 9.82
6, 3, 9.82
3, 1, 8.79
3, 2, 13.29
1, 2, 18.33
2, 4, 18.91
2, 5, 19.29
1, 2, 4.16
1, 3, 18.42
2, 3, 15.37
2, 4, 10.67
2, 5, 3.51
3, 6, 15.57
2, 1, 4.16
2, 3, 15.37
2, 4, 10.67
2, 5, 3.51
1, 3, 18.42
3, 6, 15.57
3, 1, 18.42
3, 2, 15.37
3, 6, 15.57
1, 2, 4.16
2, 4, 10.67
2, 5, 3.51
4, 2, 10.67
2, 1, 4.16
2, 3, 15.37
2, 5, 3.51
1, 3, 18.42
3, 6, 15.57
5, 2, 3.51
2, 1, 4.16
2, 3, 15.37
2, 4, 10.67
1, 3, 18.42
3, 6, 15.57
6, 3, 15.57
3, 1, 18.42
3, 2, 15.37
1, 2, 4.16
2, 4, 10.67
2, 5, 3.51
1, 2, 2.06
1, 3, 18.73
2, 3, 0.03
2, 4, 17.42
2, 5, 5.09
3, 6, 7.96
2, 1, 2.06
2, 3, 0.03
2, 4, 17.42
2, 5, 5.09
1, 3, 18.73
3, 6, 7.96
3, 1, 18.73
3, 2, 0.03
3

3, 1, 14.25
3, 5, 21.33
1, 2, 10.71
5, 7, 31.95
5, 8, 35.49
2, 4, 17.79
7, 4, 28.41
4, 6, 24.87
4, 2, 17.79
4, 6, 24.87
4, 7, 28.41
2, 1, 10.71
7, 5, 31.95
1, 3, 14.25
5, 3, 21.33
5, 8, 35.49
5, 3, 21.33
5, 7, 31.95
5, 8, 35.49
3, 1, 14.25
7, 4, 28.41
1, 2, 10.71
4, 2, 17.79
4, 6, 24.87
6, 4, 24.87
4, 2, 17.79
4, 7, 28.41
2, 1, 10.71
7, 5, 31.95
1, 3, 14.25
5, 3, 21.33
5, 8, 35.49
7, 4, 28.41
7, 5, 31.95
4, 2, 17.79
4, 6, 24.87
5, 3, 21.33
5, 8, 35.49
2, 1, 10.71
3, 1, 14.25
8, 5, 35.49
5, 3, 21.33
5, 7, 31.95
3, 1, 14.25
7, 4, 28.41
1, 2, 10.71
4, 2, 17.79
4, 6, 24.87
1, 2, 17.5
1, 3, 22.1
2, 4, 26.7
3, 5, 31.3
4, 6, 35.9
4, 7, 40.5
5, 7, 45.1
5, 8, 49.7
2, 1, 17.5
2, 4, 26.7
1, 3, 22.1
4, 6, 35.9
4, 7, 40.5
3, 5, 31.3
7, 5, 45.1
5, 8, 49.7
3, 1, 22.1
3, 5, 31.3
1, 2, 17.5
5, 7, 45.1
5, 8, 49.7
2, 4, 26.7
7, 4, 40.5
4, 6, 35.9
4, 2, 26.7
4, 6, 35.9
4, 7, 40.5
2, 1, 17.5
7, 5, 45.1
1, 3, 22.1
5, 3, 31.3
5, 8, 49.7
5, 3, 31.3
5, 7, 45.1
5, 8, 49.7
3, 1, 22.1
7, 4, 40.5
1, 2, 17.5
4, 2, 

5, 3, 28.49
3, 2, 15.89
3, 4, 24.29
2, 1, 11.69
2, 4, 20.09
4, 6, 32.69
6, 4, 32.69
4, 2, 20.09
4, 3, 24.29
2, 1, 11.69
2, 3, 15.89
3, 5, 28.49
1, 2, 12.78
2, 3, 17.53
2, 4, 22.28
3, 4, 27.03
3, 5, 31.78
4, 6, 36.53
2, 1, 12.78
2, 3, 17.53
2, 4, 22.28
3, 4, 27.03
3, 5, 31.78
4, 6, 36.53
3, 2, 17.53
3, 4, 27.03
3, 5, 31.78
2, 1, 12.78
2, 4, 22.28
4, 6, 36.53
4, 2, 22.28
4, 3, 27.03
4, 6, 36.53
2, 1, 12.78
2, 3, 17.53
3, 5, 31.78
5, 3, 31.78
3, 2, 17.53
3, 4, 27.03
2, 1, 12.78
2, 4, 22.28
4, 6, 36.53
6, 4, 36.53
4, 2, 22.28
4, 3, 27.03
2, 1, 12.78
2, 3, 17.53
3, 5, 31.78
1, 2, 6.4
2, 3, 9.19
2, 4, 11.99
3, 4, 14.78
3, 5, 17.58
4, 6, 20.38
2, 1, 6.4
2, 3, 9.19
2, 4, 11.99
3, 4, 14.78
3, 5, 17.58
4, 6, 20.38
3, 2, 9.19
3, 4, 14.78
3, 5, 17.58
2, 1, 6.4
2, 4, 11.99
4, 6, 20.38
4, 2, 11.99
4, 3, 14.78
4, 6, 20.38
2, 1, 6.4
2, 3, 9.19
3, 5, 17.58
5, 3, 17.58
3, 2, 9.19
3, 4, 14.78
2, 1, 6.4
2, 4, 11.99
4, 6, 20.38
6, 4, 20.38
4, 2, 11.99
4, 3, 14.78
2, 1, 6.4
2, 3, 9.19
3, 5, 17.58
1, 2, 13.5

6, 3, 22.4
6, 5, 25.8
3, 1, 13.9
3, 2, 15.6
3, 5, 20.7
5, 2, 19.0
5, 4, 24.1
1, 2, 12.2
2, 4, 17.3
1, 2, 13.3
1, 3, 14.47
2, 3, 15.63
2, 4, 16.8
2, 5, 17.96
3, 5, 19.12
3, 6, 20.29
4, 5, 21.45
5, 6, 22.62
2, 1, 13.3
2, 3, 15.63
2, 4, 16.8
2, 5, 17.96
1, 3, 14.47
3, 5, 19.12
3, 6, 20.29
4, 5, 21.45
5, 6, 22.62
3, 1, 14.47
3, 2, 15.63
3, 5, 19.12
3, 6, 20.29
1, 2, 13.3
2, 4, 16.8
2, 5, 17.96
5, 4, 21.45
5, 6, 22.62
4, 2, 16.8
4, 5, 21.45
2, 1, 13.3
2, 3, 15.63
2, 5, 17.96
5, 3, 19.12
5, 6, 22.62
1, 3, 14.47
3, 6, 20.29
5, 2, 17.96
5, 3, 19.12
5, 4, 21.45
5, 6, 22.62
2, 1, 13.3
2, 3, 15.63
2, 4, 16.8
3, 1, 14.47
3, 6, 20.29
6, 3, 20.29
6, 5, 22.62
3, 1, 14.47
3, 2, 15.63
3, 5, 19.12
5, 2, 17.96
5, 4, 21.45
1, 2, 13.3
2, 4, 16.8
1, 2, 15.61
1, 3, 20.41
2, 3, 25.21
2, 4, 30.01
2, 5, 34.81
3, 5, 39.61
3, 6, 44.41
4, 5, 49.21
5, 6, 54.01
2, 1, 15.61
2, 3, 25.21
2, 4, 30.01
2, 5, 34.81
1, 3, 20.41
3, 5, 39.61
3, 6, 44.41
4, 5, 49.21
5, 6, 54.01
3, 1, 20.41
3, 2, 25.21
3, 5, 39.61
3, 6, 44.41
1

3, 1, 1.61
3, 5, 0.33
3, 6, 19.22
1, 2, 4.36
5, 2, 0.59
2, 4, 19.44
4, 2, 19.44
2, 1, 4.36
2, 5, 0.59
1, 3, 1.61
5, 3, 0.33
3, 6, 19.22
5, 2, 0.59
5, 3, 0.33
2, 1, 4.36
2, 4, 19.44
3, 1, 1.61
3, 6, 19.22
6, 3, 19.22
3, 1, 1.61
3, 5, 0.33
1, 2, 4.36
5, 2, 0.59
2, 4, 19.44
1, 2, 0.77
1, 3, 13.7
2, 4, 14.72
2, 5, 3.83
3, 5, 11.61
3, 6, 1.84
2, 1, 0.77
2, 4, 14.72
2, 5, 3.83
1, 3, 13.7
5, 3, 11.61
3, 6, 1.84
3, 1, 13.7
3, 5, 11.61
3, 6, 1.84
1, 2, 0.77
5, 2, 3.83
2, 4, 14.72
4, 2, 14.72
2, 1, 0.77
2, 5, 3.83
1, 3, 13.7
5, 3, 11.61
3, 6, 1.84
5, 2, 3.83
5, 3, 11.61
2, 1, 0.77
2, 4, 14.72
3, 1, 13.7
3, 6, 1.84
6, 3, 1.84
3, 1, 13.7
3, 5, 11.61
1, 2, 0.77
5, 2, 3.83
2, 4, 14.72
1, 2, 18.61
1, 3, 13.67
2, 4, 18.26
2, 5, 2.52
3, 5, 10.01
3, 6, 3.37
2, 1, 18.61
2, 4, 18.26
2, 5, 2.52
1, 3, 13.67
5, 3, 10.01
3, 6, 3.37
3, 1, 13.67
3, 5, 10.01
3, 6, 3.37
1, 2, 18.61
5, 2, 2.52
2, 4, 18.26
4, 2, 18.26
2, 1, 18.61
2, 5, 2.52
1, 3, 13.67
5, 3, 10.01
3, 6, 3.37
5, 2, 2.52
5, 3, 10.01
2, 1, 18.61
2, 4,

6, 4, 17.1
4, 2, 58.93
4, 7, 10.94
2, 1, 13.26
7, 5, 15.89
1, 3, 54.25
5, 3, 18.67
5, 8, 52.52
7, 4, 10.94
7, 5, 15.89
4, 2, 58.93
4, 6, 17.1
5, 3, 18.67
5, 8, 52.52
2, 1, 13.26
3, 1, 54.25
8, 5, 52.52
5, 3, 18.67
5, 7, 15.89
3, 1, 54.25
7, 4, 10.94
1, 2, 13.26
4, 2, 58.93
4, 6, 17.1
1, 2, 18.2
1, 3, 56.65
2, 4, 11.78
3, 5, 17.88
4, 6, 16.29
4, 7, 11.35
5, 7, 52.94
5, 8, 12.22
2, 1, 18.2
2, 4, 11.78
1, 3, 56.65
4, 6, 16.29
4, 7, 11.35
3, 5, 17.88
7, 5, 52.94
5, 8, 12.22
3, 1, 56.65
3, 5, 17.88
1, 2, 18.2
5, 7, 52.94
5, 8, 12.22
2, 4, 11.78
7, 4, 11.35
4, 6, 16.29
4, 2, 11.78
4, 6, 16.29
4, 7, 11.35
2, 1, 18.2
7, 5, 52.94
1, 3, 56.65
5, 3, 17.88
5, 8, 12.22
5, 3, 17.88
5, 7, 52.94
5, 8, 12.22
3, 1, 56.65
7, 4, 11.35
1, 2, 18.2
4, 2, 11.78
4, 6, 16.29
6, 4, 16.29
4, 2, 11.78
4, 7, 11.35
2, 1, 18.2
7, 5, 52.94
1, 3, 56.65
5, 3, 17.88
5, 8, 12.22
7, 4, 11.35
7, 5, 52.94
4, 2, 11.78
4, 6, 16.29
5, 3, 17.88
5, 8, 12.22
2, 1, 18.2
3, 1, 56.65
8, 5, 12.22
5, 3, 17.88
5, 7, 52.94
3, 1, 56.65
7,

In [26]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [27]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [28]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 500
steps_per_epoch = len(x_train)

In [7]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    #loss2 = losses.categorical_crossentropy(y_true, y_pred)
    return loss1

In [29]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=losses.mean_squared_error, optimizer='adam')
#lstm_autoencoder_500 = lstm_autoencoder

In [30]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [31]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1)

Epoch 1/500
2352/2352 [==============================] - 22s 9ms/step - loss: 36.7292
Epoch 2/500
2352/2352 [==============================] - 20s 8ms/step - loss: 35.3402
Epoch 3/500
2352/2352 [==============================] - 20s 8ms/step - loss: 35.2929
Epoch 4/500
2352/2352 [==============================] - 20s 8ms/step - loss: 35.2844
Epoch 5/500
2352/2352 [==============================] - 20s 8ms/step - loss: 35.2721
Epoch 6/500
2352/2352 [==============================] - 20s 8ms/step - loss: 35.2665
Epoch 7/500
2352/2352 [==============================] - 20s 8ms/step - loss: 35.2638
Epoch 8/500
2352/2352 [==============================] - 20s 8ms/step - loss: 35.2584
Epoch 9/500
2352/2352 [==============================] - 20s 8ms/step - loss: 35.2625
Epoch 10/500
2352/2352 [==============================] - 20s 8ms/step - loss: 27.8085
Epoch 11/500
2352/2352 [==============================] - 20s 8ms/step - loss: 14.5989
Epoch 12/500
2352/2352 [============================

2352/2352 [==============================] - 20s 8ms/step - loss: 0.5302
Epoch 97/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.3984
Epoch 98/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.4090
Epoch 99/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.4748
Epoch 100/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.3823
Epoch 101/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.4205
Epoch 102/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.7087
Epoch 103/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.7378
Epoch 104/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.5103
Epoch 105/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.3620
Epoch 106/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.7020
Epoch 107/500
2352/2352 [==============================] - 2

2352/2352 [==============================] - 19s 8ms/step - loss: 0.1039
Epoch 191/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.1231
Epoch 192/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.1517
Epoch 193/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.1042
Epoch 194/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.1242
Epoch 195/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.1224
Epoch 196/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.1238
Epoch 197/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.1168
Epoch 198/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.1219
Epoch 199/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.1138
Epoch 200/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.1128
Epoch 201/500
2352/2352 [==============================] 

2352/2352 [==============================] - 20s 9ms/step - loss: 0.0823
Epoch 285/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0804
Epoch 286/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0775
Epoch 287/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0791
Epoch 288/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0866
Epoch 289/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0770
Epoch 290/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0818
Epoch 291/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0789
Epoch 292/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0801
Epoch 293/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0808
Epoch 294/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0701
Epoch 295/500
2352/2352 [==============================] 

2352/2352 [==============================] - 20s 8ms/step - loss: 0.0628
Epoch 379/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0584
Epoch 380/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0600
Epoch 381/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0709
Epoch 382/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0553
Epoch 383/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0675
Epoch 384/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0545
Epoch 385/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0830
Epoch 386/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0597
Epoch 387/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0528
Epoch 388/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0603
Epoch 389/500
2352/2352 [==============================] 

2352/2352 [==============================] - 19s 8ms/step - loss: 0.0533
Epoch 473/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0535
Epoch 474/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0475
Epoch 475/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0511
Epoch 476/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0524
Epoch 477/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0484
Epoch 478/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0625
Epoch 479/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0560
Epoch 480/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0493
Epoch 481/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0508
Epoch 482/500
2352/2352 [==============================] - 19s 8ms/step - loss: 0.0507
Epoch 483/500
2352/2352 [==============================] 

In [ ]:
#lstm_autoencoder_250.fit_generator(train_generator(x_train), epochs=500, steps_per_epoch=steps_per_epoch, verbose=1)#, validation_data=val_generator(x_val))

In [32]:
# model save
lstm_autoencoder.save('model/lstm_autoencoder_bfs.h5')

In [ ]:
loaded_model = load_model('model/lstm_autoencoder_bfs.h5')

In [33]:
loaded_model = lstm_autoencoder

In [34]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

1.9309339144058726


In [19]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [20]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])

In [21]:
latent_vector = np.array(latent_vector)

In [144]:
def printMatrix(matrix):
    length = int(max(max(matrix[:, 0]), max(matrix[:, 1])))
    adj = [[0 for i in range(length)] for i in range(length)]
    
    for m in matrix:
        (i, j, w) = m
        i = int(i) -1
        j = int(j) -1
        adj[i][j] = adj[j][i] = w
    for i in range(length):
        for j in range(length):
            print(str(adj[i][j]), end=',')
        print()
    print("===============")

In [213]:
# sequence to matrix
for index, x in enumerate(x_test):
    print("#", str(index))
    printMatrix(x)

# 0
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0,0,0,
0,0,18.05,0,0,0,0,32.09,0,
0,0,0,20.05,0,0,0,0,20.27,
0,0,0,0,0,32.09,0,0,9.01,
0,0,0,0,0,0,20.27,9.01,0,
# 1
0,8.79,11.79,0,0,0,
8.79,0,0,10.06,6.86,0,
11.79,0,0,0,0,7.86,
0,10.06,0,0,0,0,
0,6.86,0,0,0,0,
0,0,7.86,0,0,0,
# 2
0,3.3,6.3,0,0,
3.3,0,0,12.72,0,
6.3,0,0,0,23.87,
0,12.72,0,0,41.95,
0,0,23.87,41.95,0,
# 3
0,5.7,7.22,0,0,
5.7,0,0,12.22,0,
7.22,0,0,0,23.54,
0,12.22,0,0,42.39,
0,0,23.54,42.39,0,
# 4
0,12.52,17.52,28.77,0,0,0,0,0,
12.52,0,0,0,0,0,0,0,0,
17.52,0,0,0,2.53,4.53,0,0,0,
28.77,0,0,0,0,0,10.86,0,0,
0,0,2.53,0,0,0,0,0,0,
0,0,4.53,0,0,0,0,9.01,0,
0,0,0,10.86,0,0,0,0,10.01,
0,0,0,0,0,9.01,0,0,20.01,
0,0,0,0,0,0,10.01,20.01,0,
# 5
0,4.27,9.29,0,0,
4.27,0,0,12.29,0,
9.29,0,0,0,17.29,
0,12.29,0,0,3.95,
0,0,17.29,3.95,0,
# 6
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0

In [171]:
standard = latent_vector[0]

In [223]:
import copy
dist = []
left = 100000
right = -10000
for index, v in enumerate(latent_vector[1:]):
    d = round(euclidean_distance(standard, v), 8)
    dist.append(d)
    if d > right:
        right= d
    if left > d:
        left = d
dist1 = copy.deepcopy(dist)
dist1.sort()
#print(dist, dist1)
for index, d in enumerate(dist1):
    print( (index+1, dist.index(d)+1), end='  ')

(1, 2)  (2, 38)  (3, 53)  (4, 5)  (5, 15)  (6, 44)  (7, 48)  (8, 21)  (9, 12)  (10, 43)  (11, 9)  (12, 18)  (13, 31)  (14, 59)  (15, 58)  (16, 4)  (17, 41)  (18, 27)  (19, 29)  (20, 33)  (21, 6)  (22, 3)  (23, 51)  (24, 35)  (25, 54)  (26, 55)  (27, 26)  (28, 62)  (29, 14)  (30, 46)  (31, 11)  (32, 45)  (33, 7)  (34, 50)  (35, 57)  (36, 8)  (37, 63)  (38, 60)  (39, 24)  (40, 52)  (41, 20)  (42, 13)  (43, 37)  (44, 23)  (45, 10)  (46, 1)  (47, 61)  (48, 42)  (49, 56)  (50, 34)  (51, 16)  (52, 32)  (53, 40)  (54, 39)  (55, 19)  (56, 17)  (57, 22)  (58, 49)  (59, 30)  (60, 36)  (61, 47)  (62, 28)  (63, 25)  

In [168]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum(a-b) ** 2)